In [27]:
from os import getcwd

with open(getcwd() + "/LWs/LW_8/data/amazon_cells_labelled.txt") as file_amazon:
    data = file_amazon.readlines()

with open(getcwd() + "/LWs/LW_8/data/imdb_labelled.txt") as file_imdb:
    temp = file_imdb.readlines()
    data = data + temp

with open(getcwd() + "/LWs/LW_8/data/yelp_labelled.txt") as file_yelp:
    temp = file_yelp.readlines()
    data = data + temp


In [29]:
x = []
y = []
counter = 0

for value in data:
    temp = value.split('\t')
    x.append(temp[0])
    temp[1].replace('\n','')
    y.append(int(temp[1]))
    if counter < 15:
        print(temp[0], "\n", temp[1])
    counter += 1
print ("...")
# 0 - negative review, 1- positive review

So there is no way for me to plug it in here in the US unless I go by a converter. 
 0

Good case, Excellent value. 
 1

Great for the jawbone. 
 1

Tied to charger for conversations lasting more than 45 minutes.MAJOR PROBLEMS!! 
 0

The mic is great. 
 1

I have to jiggle the plug to get it to line up right to get decent volume. 
 0

If you have several dozen or several hundred contacts, then imagine the fun of sending each of them one by one. 
 0

If you are Razr owner...you must have this! 
 1

Needless to say, I wasted my money. 
 0

What a waste of money and time!. 
 0

And the sound quality is great. 
 1

He was very impressed when going from the original battery to the extended battery. 
 1

If the two were seperated by a mere 5+ ft I started to notice excessive static and garbled sound from the headset. 
 0

Very good quality though 
 1

The design is very odd, as the ear "clip" is not very comfortable at all. 
 0

...


In [30]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=2500,split=' ')
tokenizer.fit_on_texts(x)
print(tokenizer.word_index)


{'pleasant': 1015, 'salmon': 1068, 'pure': 1274, 'spicier': 4835, 'vacant': 4911, 'compete': 2677, 'related': 3808, 'theatres': 3475, 'weeks': 765, 'weight': 1557, 'deadpan': 4206, 'applauded': 3646, 'masterful': 1861, 'carry': 1723, 'produce': 1666, 'accomodate': 4938, 'infuriating': 2657, 'expectations': 1132, 'predictably': 1807, 'involved': 852, 'glance': 1378, 'alert': 4384, 'earphones': 2524, 'efforts': 3017, 'continuation': 3290, 'rpger': 4037, 'having': 391, 'fulfilling': 3543, 'lovable': 3337, 'derivative': 3597, 'data': 785, 'ipod': 1585, 'instruction': 2454, 'fellow': 4724, 'flavored': 4611, 'accused': 1786, 'animated': 3944, 'saving': 4917, 'selection': 470, 'slid': 2631, 'la': 3593, 'greater': 2318, 'reviewer': 1355, 'vegan': 4939, 'row': 1710, 'theater': 1026, 'need': 407, 'negative': 1283, 'hits': 4639, 'waitresses': 4624, 'following': 1799, 'perfection': 2223, 'rings': 4992, 'thirty': 4910, 'possible': 629, 'pineapple': 5014, 'consumer': 1560, 'indication': 4266, 'rge':

In [31]:
from keras.preprocessing.sequence import pad_sequences

X = tokenizer.texts_to_sequences(x)
X = pad_sequences(X)
X

array([[   0,    0,    0, ...,   60,    4, 2267],
       [   0,    0,    0, ...,  157,   91,  526],
       [   0,    0,    0, ...,   12,    1, 1093],
       ...,
       [   0,    0,    0, ...,   13,   75,   69],
       [   0,    0,    0, ...,  368,  299,   40],
       [   0,    0,    0, ...,  627,    1,  488]], dtype=int32)

In [32]:
import numpy as np 

Y = []
for val in y:
    if(val == 0):
        Y.append([1,0])
    else:
        Y.append([0,1])
Y = np.array(Y)
Y

array([[1, 0],
       [0, 1],
       [0, 1],
       ...,
       [1, 0],
       [1, 0],
       [1, 0]])

In [33]:
print(X.shape)

(3000, 61)


In [34]:
import keras 
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(Embedding(2500,128,input_length=X.shape[1],dropout=0.2))
model.add(LSTM(300, dropout_U=0.2,dropout_W=0.2))
model.add(Dense(2,activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer='adam',metrics=['accuracy'])

print(model.summary())

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(300, dropout=0.2, recurrent_dropout=0.2)`
  import sys


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 61, 128)           320000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               514800    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 602       
Total params: 835,402
Trainable params: 835,402
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [36]:

model.fit(x_train,y_train,epochs=10,verbose=2,batch_size=32)

Epoch 1/10


 - 21s - loss: 0.6821 - acc: 0.6317


Epoch 2/10


 - 23s - loss: 0.4531 - acc: 0.8221


Epoch 3/10


...

 - 28s - loss: 0.0336 - acc: 0.9921


Epoch 9/10


 - 31s - loss: 0.0324 - acc: 0.9929


Epoch 10/10


 - 26s - loss: 0.0238 - acc: 0.9950


In [39]:
print("RESLUT:", "\n", model.evaluate(x_test,y_test)[1] * 100, "%")

 32/600 [>.............................] - ETA: 1s

 64/600 [==>...........................] - ETA: 1s

 96/600 [===>..........................] - ETA: 1s

128/600 [=====>........................] - ETA: 1s

160/600 [=======>......................] - ETA: 1s

...

512/600 [========================>.....] - ETA: 0s

544/600 [==========================>...] - ETA: 0s

576/600 [===========================>..] - ETA: 0s

600/600 [==============================] - 1s 2ms/step


RESLUT: 
 82.3333334128062 %
